In [1]:
import folium
import pandas as pd

# read data
file_path = '/Users/kouchenlu/Desktop/Visualisation_Preferred Strategy.xlsx'

data_sources = pd.read_excel(file_path, sheet_name='Source')
data_factories = pd.read_excel(file_path, sheet_name='Factory')
data_ports = pd.read_excel(file_path, sheet_name='Port')

data_f_all = pd.read_excel(file_path, sheet_name='Factory Allocation')
data_p_all = pd.read_excel(file_path, sheet_name='Port Allocation')

data = {
    'Source': data_sources,
    'Factory': data_factories,
    'Port': data_ports
}

# create a map
center_lat = (data_sources['Latitude'].mean() + data_factories['Latitude'].mean() + data_ports['Latitude'].mean()) / 3
center_lon = (data_sources['Longitude'].mean() + data_factories['Longitude'].mean() + data_ports['Longitude'].mean()) / 3

mymap = folium.Map(location=[center_lat, center_lon], zoom_start=5)


In [2]:
# create layers - Source, Factory and Port
source_layer = folium.FeatureGroup(name='Source')
factory_layer = folium.FeatureGroup(name='Factory')
port_layer = folium.FeatureGroup(name='Port')

# layer: Source
for _, row in data_sources.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=4,
        color='darksalmon',
        fill=True,
        fill_color='darksalmon',
        fill_opacity=0.7,
        popup=f"""
        <strong>Town:</strong> {row.get('Town', 'N/A')}<br>
        <strong>Description:</strong> {row.get('Description', 'N/A')}<br>
        <strong>Production:</strong> {row.get('Production', 'N/A')}
        """,
        tooltip=row['Town']
    ).add_to(source_layer)
    
# layer: Factory
for _, row in data_factories.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=6,
        color='darkcyan',
        fill=True,
        fill_color='darkcyan',
        fill_opacity=0.7,
        popup=f"""
        <strong>Town:</strong> {row.get('Town', 'N/A')}<br>
        <strong>Description:</strong> {row.get('Description', 'N/A')}<br>
        <strong>Quantity:</strong> {row.get('Quantity', 'N/A')}
        """,
        tooltip=row['Town']
    ).add_to(factory_layer)

# layer: Port
for _, row in data_ports.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=7,
        color='firebrick',
        fill=True,
        fill_color='firebrick',
        fill_opacity=0.7,
        popup=f"""
        <strong>Town:</strong> {row.get('Town', 'N/A')}<br>
        <strong>Description:</strong> {row.get('Description', 'N/A')}<br>
        <strong>Quantity:</strong> {row.get('Quantity', 'N/A')}
        """,
        tooltip=row['Town']
    ).add_to(port_layer)

In [3]:
# create layers - Factory and Port Allocation



# Factory Allocation
categories_f = data_f_all['Category'].unique()
category_layers_f = {}

for category in categories_f:
    category_layers_f[category] = folium.FeatureGroup(name=f'Factory Allocation - {category}')
    mymap.add_child(category_layers_f[category])

# find allocations
for _, fac_allocation in data_f_all.iterrows():
    town1_name = fac_allocation['Source']
    town2_name = fac_allocation['Factory']
    category = fac_allocation['Category']
    
    mode = fac_allocation['Mode']
    q = fac_allocation['Production']
    
    # find town1
    town1_location = None
    for key, df in data.items():
        town1_row = df[df['Town'] == town1_name]
        if not town1_row.empty:
            town1_location = [town1_row.iloc[0]['Latitude'], town1_row.iloc[0]['Longitude']]
            break
    
    # find town2
    town2_location = None
    for key, df in data.items():
        town2_row = df[df['Town'] == town2_name]
        if not town2_row.empty:
            town2_location = [town2_row.iloc[0]['Latitude'], town2_row.iloc[0]['Longitude']]
            break
    
    # draw lines
    if town1_location and town2_location:
        popup_f_all=f"""
        <strong>Source:</strong> {town1_name}<br>
        <strong>Factory:</strong> {town2_name}<br>
        <strong>Mode:</strong> {mode}<br>
        <strong>Quantity:</strong> {q}
        """
        folium.PolyLine(
            locations=[town1_location, town2_location],
            color = 'steelblue' if category == 'Water' else 'chocolate',
            weight=1.25, 
            opacity=0.8, 
            popup=popup_f_all
        ).add_to(category_layers_f[category])

        

# Port Allocation
categories_p = data_p_all['Category'].unique()
category_layers_p = {}

for category in categories_p:
    category_layers_p[category] = folium.FeatureGroup(name=f'Port Allocation - {category}')
    mymap.add_child(category_layers_p[category])   
        
# find allocations
for _, port_allocation in data_p_all.iterrows():
    town1_name = port_allocation['Factory']
    town2_name = port_allocation['Port']
    category = port_allocation['Category']
    
    mode = port_allocation['Mode']
    q = port_allocation['Quantity']
    
    # find town1
    town1_location = None
    for key, df in data.items():
        town1_row = df[df['Town'] == town1_name]
        if not town1_row.empty:
            town1_location = [town1_row.iloc[0]['Latitude'], town1_row.iloc[0]['Longitude']]
            break
    
    # find town2
    town2_location = None
    for key, df in data.items():
        town2_row = df[df['Town'] == town2_name]
        if not town2_row.empty:
            town2_location = [town2_row.iloc[0]['Latitude'], town2_row.iloc[0]['Longitude']]
            break
    
    # draw lines
    if town1_location and town2_location:
        popup_p_all=f"""
        <strong>Factory:</strong> {town1_name}<br>
        <strong>Port:</strong> {town2_name}<br>
        <strong>Mode:</strong> {mode}<br>
        <strong>Quantity:</strong> {q}
        """
        folium.PolyLine(
            locations=[town1_location, town2_location],
            color = 'steelblue' if category == 'Water' else 'chocolate',
            weight=3, 
            opacity=0.8, 
            popup=popup_p_all
        ).add_to(category_layers_p[category])

In [4]:
# add layers to the map

source_layer.add_to(mymap)
factory_layer.add_to(mymap)
port_layer.add_to(mymap)

for category_layer in category_layers_f.values():
    category_layer.add_to(mymap)
    
for category_layer in category_layers_p.values():
    category_layer.add_to(mymap)

folium.LayerControl().add_to(mymap)

# save as html
mymap.save('Map_Preferred Strategy.html')
